In [1]:
import pandas as pd 
from utilities.utils import * 
import ast 

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jannis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
full_df = pd.read_csv('../../data_files/pipeline_steps/6.1_sentences_filtersadded.csv')

In [3]:
len(full_df) 

346682

In [4]:
full_df.head() 

,Unnamed: 0,article_index,resolved_sentence_list,sentence_list,pos_sentence_list,pos_resolved_sentence_list,quot_truth_list,link_truth_list,linkname_truth_list,pos_sentence_strings,pos_resolved_sentence_strings
0,0,0,nan,NaN,[],['X'],False,True,True,NaN,X
1,1,1,"René ist ein männlicher Vorname , der auch als...","René ist ein männlicher Vorname, der auch als ...","['PROPN', 'ADJ', 'ADJ', 'NOUN', 'VERB']","['PROPN', 'ADJ', 'NOUN', 'SPACE', 'NOUN', 'VERB']",False,True,True,PROPN ADJ ADJ NOUN VERB,PROPN ADJ NOUN SPACE NOUN VERB
2,2,2,Depp ist : Depp ist der Familienname folgender...,Depp ist: Depp ist der Familienname folgender ...,"['NOUN', 'AUX', 'NOUN', 'NOUN', 'ADJ', 'NOUN',...","['NOUN', 'SPACE', 'NOUN', 'NOUN', 'ADJ', 'NOUN...",False,False,False,NOUN AUX NOUN NOUN ADJ NOUN VERB ADV,NOUN SPACE NOUN NOUN ADJ NOUN SPACE VERB
3,3,3,HAVA oder Hava steht als Abkürzung oder Künstl...,HAVA oder Hava steht als Abkürzung oder Künstl...,"['PROPN', 'PROPN', 'VERB', 'ADJ', 'NOUN', 'ADP...","['PROPN', 'PROPN', 'VERB', 'NOUN', 'NOUN', 'SP...",False,False,False,PROPN PROPN VERB ADJ NOUN ADP VERB ADV,PROPN PROPN VERB NOUN NOUN SPACE VERB
4,4,4,SPD steht für : Parteien : SPD steht für : Spd...,SPD steht für: Parteien: SpD steht für: Spd st...,"['PROPN', 'VERB', 'ADP', 'NOUN', 'PROPN', 'VER...","['PROPN', 'VERB', 'SPACE', 'NOUN', 'SPACE', 'P...",False,False,False,PROPN VERB ADP NOUN PROPN VERB ADP PROPN VERB...,PROPN VERB SPACE NOUN SPACE PROPN VERB SPACE ...


In [12]:
full_df.dtypes 

Unnamed: 0                        int64
article_index                     int64
resolved_sentence_list           object
sentence_list                    object
pos_sentence_list                object
pos_resolved_sentence_list       object
quot_truth_list                    bool
link_truth_list                    bool
linkname_truth_list                bool
pos_sentence_strings             object
pos_resolved_sentence_strings    object
dtype: object

In [5]:
print(full_df.columns)

Index(['Unnamed: 0.1', 'Unnamed: 0', 'article_index', 'nopos_resolved_text',
       'nopos_nonresolved_text', 'pos_nonresolved_text', 'pos_resolved_text',
       'quot_label', 'link_label', 'namelink_label',
       'pos_nonresolved_quot_filter', 'pos_nonresolved_link_filter',
       'pos_nonresolved_namelink_filter', 'pos_resolved_quot_filter',
       'pos_resolved_link_filter', 'pos_resolved_namelink_filter'],
      dtype='object')


In [20]:
df=full_df 

In [29]:

rename = {
    'resolved_sentence_list': 'nopos_resolved_text',
    'sentence_list': 'nopos_nonresolved_text',
    'quot_truth_list': 'quot_label',
    'link_truth_list': 'link_label',
    'linkname_truth_list': 'namelink_label',
    'pos_sentence_strings': 'pos_nonresolved_text',
    'pos_resolved_sentence_strings': 'pos_resolved_text',
}


## Equal & Random

In [25]:
sample= 25000
labels = ['quot_label', 'link_label', 'namelink_label']
texts = ['nopos_resolved_text', 'nopos_nonresolved_text', 'pos_nonresolved_text', 'pos_resolved_text']
if_filter = ['filter','nofilter']

df_equal_sample = df.copy()
df_equal_sample = df_equal_sample.sample(n=(sample*2))

for label in labels:
    df_random = pd.DataFrame()
    df_equal = pd.DataFrame()

    for text in texts:
        for to_filter in if_filter:


            name =f"{text[:-5]}_{label[:-6]}_{to_filter}"
            if to_filter == 'filter' and not text[:3] == 'pos':
                name = f"{text[2:-5]}_{label[:-6]}_filter"
            df_filter_random = df.copy()
            df_filter_equal = df_equal_sample.copy()
            if to_filter == 'filter':
                df_filter_random = df_filter_random.query(f'not ({label} == False and {name} == True)')
                df_filter_equal = df_filter_equal.query(f'not ({label} == False and {name} == True)')
                name = f"{text[:-5]}_{label[:-6]}_{to_filter}"
            df_random_grouped = df_filter_random.groupby(label)
            df_random_sample = df_random_grouped.apply(lambda x: x.sample(n=sample)).reset_index(drop=True)

            df_random['label'] = df_random_sample[label]
            df_equal['label'] = df_equal_sample[label]

            df_random['text'] = ''
            df_equal['text'] = ''


            df_random['text'] = df_random_sample[text]
            df_equal['text'] = df_equal_sample[text]

            df_random = df_random.sample(frac=1)
            df_equal = df_equal.sample(frac=1)

            df_random.to_csv(f'../../data_files/test_samples/3rd_test_dataset_samples/{name}_random.csv')
            df_equal.to_csv(f'../../data_files/test_samples/3rd_test_dataset_samples/{name}_equal.csv')


## Only Equal

In [24]:
sample = 25
labels = ['quot_label', 'link_label', 'namelink_label']
texts = [
    'nopos_resolved_text', 'nopos_nonresolved_text', 'pos_nonresolved_text', 'pos_resolved_text'
]
if_filter = ['filter', 'nofilter']

df_equal_sample = df.copy()
df_equal_sample = df_equal_sample.sample(n=(sample * 2))

list =[]


for label in labels:
    df_equal = pd.DataFrame()

    for text in texts:
        for to_filter in if_filter:

            name = f"{text[:-5]}_{label[:-6]}_{to_filter}"
            df_filter_equal = df_equal_sample.copy()

            if to_filter == 'filter' and not text[:3] == 'pos':
                name = f"{text[2:-5]}_{label[:-6]}_filter"
                
            if to_filter == 'filter':
                df_filter_equal = df_filter_equal.query(f'not ({label} == False and {name} == True)')
                name = f"{text[:-5]}_{label[:-6]}_{to_filter}"
            df_equal['label'] = df_equal_sample[label]

            df_equal['text'] = ''

            df_equal['text'] = df_equal_sample[text]

            df_equal = df_equal.sample(frac=1)

            df_equal.to_csv(f'../../data_files/test_samples/3rd_test_dataset_samples/{name}_equal.csv')
            list.append(f'{name}_equal.csv')



In [16]:
print(list)

['nopos_resolved_quot_filter_equal.csv', 'nopos_resolved_quot_nofilter_equal.csv', 'nopos_nonresolved_quot_filter_equal.csv', 'nopos_nonresolved_quot_nofilter_equal.csv', 'pos_nonresolved_quot_filter_equal.csv', 'pos_nonresolved_quot_nofilter_equal.csv', 'pos_resolved_quot_filter_equal.csv', 'pos_resolved_quot_nofilter_equal.csv', 'nopos_resolved_link_filter_equal.csv', 'nopos_resolved_link_nofilter_equal.csv', 'nopos_nonresolved_link_filter_equal.csv', 'nopos_nonresolved_link_nofilter_equal.csv', 'pos_nonresolved_link_filter_equal.csv', 'pos_nonresolved_link_nofilter_equal.csv', 'pos_resolved_link_filter_equal.csv', 'pos_resolved_link_nofilter_equal.csv', 'nopos_resolved_namelink_filter_equal.csv', 'nopos_resolved_namelink_nofilter_equal.csv', 'nopos_nonresolved_namelink_filter_equal.csv', 'nopos_nonresolved_namelink_nofilter_equal.csv', 'pos_nonresolved_namelink_filter_equal.csv', 'pos_nonresolved_namelink_nofilter_equal.csv', 'pos_resolved_namelink_filter_equal.csv', 'pos_resolved_n